In [1]:
%load_ext autoreload
%autoreload 2

In [45]:
import matplotlib as mpl
from matplotlib import rc
rc('text',usetex=False)
rc('font',**{'family':'serif','serif':['Times New Roman'],'size':8})
rc('mathtext',fontset='cm')
import matplotlib.pyplot as plt
% matplotlib inline
% config InlineBackend.figure_format = 'svg'
% config InlineBackend.rc = {'figure.figsize': (5,3.5)}

In [25]:
import numpy as np
from numpy import sqrt,log,array as ar
import numpy.linalg as la
import cvxpy as cvx
from helper import *
from cd.model.utility import LipschitzExpUtility as leu, RiskNeutralUtility as rnu
from cd.model.distrs import RademacherDistribution,UniformDistribution
import cd.model.synth_data as synth
import scaler
from itertools import product,zip_longest
from scipy.optimize import curve_fit

In [109]:
fig,axarr = plt.subplots(2,sharex=True,figsize=(6,8))
pltu,pltce = axarr
pltu.plot(ps,error[:,2],'C0-o',ms=4,markevery=4,label='Aucune information')
pltu.plot(ps,error[:,1],'C0-s',ms=4,markevery=4,label="Information dispers\\'ee")
pltu.plot(ps,error[:,0],'C0',label="Information concentr\\'ee")
pltu.plot(ps[0],error[0,0],'k--',label='Borne (commune)')
pltu.tick_params('y',colors='C0')
pltu.legend(loc='upper left')
pltu.set_title("a) Utilit\\'e")
yo = pltu.get_yticks()
pltu.set_yticks([el for el in yo if el <= error.max()]);
pltu.axis(ymax=.6,ymin=error.min()-0.025)

pltu2 = pltu.twinx()
pltu2.plot(ps,bound[:,0],'k--')
pltu2.axis(ymin=-30)
yo = pltu2.get_yticks()
pltu2.set_yticks([el for el in yo if el >= bound.min()]+[0]);

pltce.plot(ps,errorce[:,2],'C0-o',markevery=4,ms=4,label='Aucune information')
pltce.plot(ps,errorce[:,1],'C0-s',markevery=4,ms=4,label="Information dispers\\'ee")
pltce.plot(ps,errorce[:,0],'C0',label='Information concentree')
pltce.plot(ps[0],errorce[0,2],'k--o',markevery=4,ms=4,label='Borne',zorder=-1)
pltce.plot(ps[0],errorce[0,1],'k--s',markevery=4,ms=4,label="Borne",zorder=-1)
pltce.plot(ps[0],errorce[0,0],'k--',label='Borne',zorder=-1)

pltce.legend(loc='upper left',ncol=2)
pltce.set_title("b) \\'Equivalent certain")
pltce.set_xlabel('$p$')
yo = pltce.get_yticks()
pltce.set_yticks([el for el in yo if el <= error.max()]+[0.3]);
pltce.axis(ymax=.7,ymin=error.min()-.01)
pltce.tick_params('y',colors='C0')

pltce2 = pltce.twinx()
pltce2.plot(ps,boundce[:,0],'k--',label="Information concentr\\'ee")
pltce2.plot(ps,boundce[:,2],'k--o',ms=4,markevery=4,label="Aucune information")
pltce2.plot(ps[1:],boundce[1:,1],'k--s',ms=4,markevery=4,label="Information dispers\\'ee")
pltce2.axis(ymin=-35)
yo = pltce2.get_yticks()
pltce2.set_yticks([el for el in yo if el >= error.min()]);

# plt.savefig('fig/pconst_infogen.pdf',bbox_inches='tight',pad_inches=0)



In [110]:
u = leu(1)
ps = np.arange(1,51)
R = RademacherDistribution()
Xs = [RademacherDistribution() for _ in range(max(ps))]

In [111]:
corvecs = np.empty((3,len(ps)))
corvecs[0] = [1]+[0]*(len(ps)-1) # Info concentree
corvecs[1] = [-0.00001+1/sqrt(max(ps))]*len(ps)
corvecs[2] = 0

In [112]:
Ms = [synth.GaussianMarket(Xs,R,corr_vector=corvec) for corvec in corvecs]
t_trues = [M.sample_t(1_000) for M in Ms]

In [113]:
q_trues = [solve(t_true,u=u,lamb=0) for t_true in t_trues]
eu_trues = [u(t_true@q_true[0]).mean() for t_true,q_true in zip(t_trues,q_trues)]

In [114]:
m = 100
n = 100
tss = [create_sample(t_true,n=n,m=m) for t_true in t_trues]

In [115]:
k = 3
error = np.zeros((len(ps),k))
errorce = np.zeros((len(ps),k))
for i,p in enumerate(ps):
    if i%5 == 0: print(i)
    for j,(ts,t_true,eu_true) in enumerate(zip(tss,t_trues,eu_trues)):
        t = ts[:,:,:p]
        qs = solve(t,u,lamb=1)
        outerror = out_error(qs,u,t_true,lamb=1)
        errorce[i,j] = np.percentile(u.inverse(eu_true) - u.inverse(outerror),95)
        # errorce[i,j] = np.max(u.inverse(eu_true) - u.inverse(outerror))
        error[i,j] = np.percentile(eu_true - outerror,95)
        # error[i,j] = np.max(eu_true - outerror)


0


5


10


15


20


25


30


35


40


45


In [132]:
fig,axarr = plt.subplots(2,sharex=True,figsize=(6,8))
pltu,pltce = axarr
pltu.plot(ps,error[:,0],'C0',label="Information concentr\\'ee")
pltu.plot(ps,error[:,1],'C0-s',ms=4,markevery=4,label="Information dispers\\'ee")
pltu.plot(ps,error[:,2],'C0-o',ms=4,markevery=4,label='Aucune information')
pltu.axis(ymax=0.9)
pltu.set_title("a) Utilit\\'e")
pltu.legend(loc='upper left')

# 
pltce.plot(ps[0],errorce[0,2],'C0',ms=4,markevery=4,label="Info. concentr\\'ee")
pltce.plot(ps,errorce[:,1],'C0-s',ms=4,markevery=4,label="Info. dispers\\'ee")
pltce.plot(ps,errorce[:,2],'C0-o',ms=4,markevery=4,label='Aucune information')
pltce.set_title("b) \\'Equivalent certain")
pltce.set_xlabel('$p$')
pltce.axis(ymax=1.2)
# pltce.tick_params('y',colors='C0')

yo = pltce.get_yticks()
pltce.set_yticks([el for el in yo if el <= errorce[:,1].max()])
pltce.axis(ymin=-0.1)
pltce2 = pltce.twinx()
pltce2.plot(ps,errorce[:,0],'C0')
pltce2.axis(ymax=21.96,ymin=21.5)
yo = pltce2.get_yticks()
pltce2.set_yticks([el for el in yo if el >= errorce[:,0].min()]+[21.95,21.85,21.8]);

pltce.legend(loc='upper left')

# plt.savefig('fig/pconst_infosototale.pdf',bbox_inches='tight',pad_inches=0)



In [175]:
t_trues[0],eu_trues[0],q_trues[0]

(array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 1., -1.,  1., ...,  1., -1.,  1.],
        [ 1., -1.,  1., ..., -1.,  1.,  1.],
        ..., 
        [ 1., -1.,  1., ..., -1.,  1., -1.],
        [ 1., -1., -1., ...,  1.,  1., -1.],
        [ 1.,  1., -1., ...,  1., -1.,  1.]]),
 0.99999999987127386,
 array([[  2.27733307e+01,  -1.71628820e-05,  -7.40275436e-06,
          -2.62200384e-05,  -8.13979375e-06,   1.10319892e-05,
           9.02648981e-06,   6.95866246e-06,  -1.21153477e-05,
           1.64728988e-05]]))

In [133]:
u = leu(1)
ps = np.arange(1,51)
R = RademacherDistribution()
Xs = [RademacherDistribution() for _ in range(max(ps))]

In [134]:
corvecs = np.empty((3,len(ps)))
corvecs[0] = [1]+[0]*(len(ps)-1) # Info concentree
corvecs[1] = [-0.00001+1/sqrt(max(ps))]*len(ps)
corvecs[2] = 0

In [135]:
Ms = [synth.GaussianMarket(Xs,R,corr_vector=corvec) for corvec in corvecs]
t_trues = [M.sample_t(1_000) for M in Ms]

In [138]:
q_trues = [[solve(t_true[:,:p],u=u,lamb=0)[0] for p in ps] for t_true in t_trues]
q_trues = np.array([np.array(list(zip_longest(*q_true,fillvalue=0))).T for q_true in q_trues])

In [140]:
eu_trues = np.array([[u(t_true@q_true).mean() for q_true in q_true0] for q_true0,t_true in zip(q_trues,t_trues)])

In [143]:
plt.plot(ps,eu_trues[0,:],'C0',label="Information concentr\\'ee")
plt.plot(ps,eu_trues[1,:],'C0-s',ms=4,markevery=4,label="Information dispers\\'ee")
plt.plot(ps,eu_trues[2,:],'C0-o',ms=4,markevery=4,label='Aucune information')
plt.axis(ymax=1.4)
plt.legend()
plt.xlabel('$p$')
plt.savefig('fig/pconst_euinforelative.pdf',bbox_inches='tight',pad_inches=0)


In [155]:
m = 100
n = 100
tss = [create_sample(t_true,n=n,m=m) for t_true in t_trues]

In [157]:
k = 3
error = np.zeros((len(ps),k))
bound = np.zeros((len(ps),k))
errorce = np.zeros((len(ps),k))
for i,p in enumerate(ps):
    if i%5 == 0: print(i)
    for j,(ts,t_true,q_true) in enumerate(zip(tss,t_trues,q_trues)):
        t = ts[:,:,:p]
        qs = solve(t,u,lamb=1)
        outerror = out_error(qs,u,t_true,lamb=1)
        errorce[i,j] = np.percentile(u.inverse(eu_trues[j,i]) - u.inverse(outerror),95)
        error[i,j] = np.percentile(eu_trues[j,i] - outerror,95)
    

0


5


10


15


20


25


30


35


40


45


In [193]:
q_trues.shape

(3, 50, 50)

In [206]:
bound = np.zeros((len(ps),k))
for i,p in enumerate(ps):
    if i%5 == 0: print(i)
    for j,q_true in enumerate(q_trues):
        bound[i,j] = bound_sou(n,sqrt(p),q_true[j])
        

0
5
10
15
20
25
30
35
40
45


In [207]:
plt.plot(bound)

In [200]:
q_true[0]

(50,)

In [167]:
fig,axarr = plt.subplots(2,sharex=True,figsize=(6,8))
pltu,pltce = axarr
pltu.plot(ps,error[:,0],'C0',label="Information concentr\\'e")
pltu.plot(ps,error[:,1],'C0-s',ms=4,markevery=4,label="Information dispers\\'ee")
pltu.plot(ps,error[:,2],'C0-o',ms=4,markevery=4,label='Aucune information')
pltu.set_title("a) Utilit\\'e")
pltu.axis(ymax=0.88)
pltu.legend(loc='upper left')

pltce.plot(ps[0],errorce[0,2],'C0',ms=4,markevery=4,label="Info. concentr\\'ee")
pltce.plot(ps,errorce[:,1],'C0-s',ms=4,markevery=4,label="Info. dispers\\'ee")
pltce.plot(ps,errorce[:,2],'C0-o',ms=4,markevery=4,label='Aucune info.')
pltce.set_title("b) \\'Equivalent certain")
pltce.set_xlabel('$p$')
pltce.axis(ymax=1.2)

yo = pltce.get_yticks()
pltce.set_yticks([el for el in yo if el <= errorce[:,1].max()]+[0.6])
pltce.axis(ymin=-0.1)

pltce2 = pltce.twinx()
pltce2.plot(ps,errorce[:,0],'C0')
pltce2.axis(ymin=12,ymax=28)
yo = pltce2.get_yticks()
# pltce2.set_yticks([el for el in yo if el >= errorce[:,0].min()]+[20]);
pltce2.set_yticks([21,22,23,24,25])
pltce.axis(ymax=1.2)
pltce.legend(loc='upper left',fontsize=9.5)

# plt.savefig('fig/pconst_infosorelative.pdf',bbox_inches='tight',pad_inches=0)



In [210]:
plt.plot(ps,error[:,0])

In [263]:
u = leu(1)
ps = np.arange(1,51)
R = RademacherDistribution()
Xs = [RademacherDistribution() for _ in range(max(ps))]

In [217]:
corvecs = np.empty((3,len(ps)))
corvecs[0] = [1]+[0]*(len(ps)-1) # Info concentree
corvecs[1] = [-0.00001+1/sqrt(max(ps))]*len(ps)
corvecs[2] = 0

In [218]:
Ms = [synth.GaussianMarket(Xs,R,corr_vector=corvec) for corvec in corvecs]
t_trues = [M.sample_t(1_000) for M in Ms]

In [219]:
q_trues = [[solve(t_true[:,:p],u=u,lamb=0)[0] for p in ps] for t_true in t_trues]
q_trues = np.array([np.array(list(zip_longest(*q_true,fillvalue=0))).T for q_true in q_trues])

Combined backtracking failed 0 0 90 0 sigma 1


In [220]:
eu_trues = np.array([[u(t_true@q_true).mean() for q_true in q_true0] for q_true0,t_true in zip(q_trues,t_trues)])

In [225]:
m = 100
n = 100
tss = [create_sample(t_true,n=n,m=m) for t_true in t_trues]

In [265]:
k = 3
error = np.zeros((len(ps),k))
bound = np.zeros((len(ps),k))
errorce = np.zeros((len(ps),k))
boundce = np.zeros((len(ps),k))
for i,p in enumerate(ps):
    if i%5 == 0: print(i)
    for j,(ts,t_true,q_true) in enumerate(zip(tss,t_trues,q_trues)):
        t = ts[:,:,:p]
        qs = solve(t,u,lamb=1)
        outerror = out_error(qs,u,t_true,lamb=1)
        errorce[i,j] = np.percentile(u.inverse(eu_trues[j,i]) - u.inverse(outerror),95)
        error[i,j] = np.percentile(eu_trues[j,i] - outerror,95)
        bound[i,j] = bound_sou(n,sqrt(p),q_true[j])
        boundce[i,j] = bound[i,j] * u.subinverse(u.inverse(eu_trues[j,i]))
    

0


5


10


15


20


25


30


35


40


45


In [322]:
plt.figure(figsize=(10,20))
fig,ax = plt.subplots(2,sharex=True,figsize=(6,8))
pltu,pltce = ax
pltu.plot(ps,error[:,2],label="Aucune information")
pltu.plot(ps[0],error[0,2],'k--',label='Borne')
yo = pltu.get_yticks()
pltu.set_yticks([el for el in yo if el <= error[:,2].max()])
pltu.tick_params('y',colors='C0')
pltu.axis(ymax=0.18,ymin=0)


pltu2 = pltu.twinx()
pltu2.plot(ps,bound[:,2],'k--')
pltu2.axis(ymin=-300)
yo = pltu2.get_yticks()
pltu2.set_yticks([el for el in yo if el >= bound[:,2].min()]+[0])
pltu.legend(loc='upper left')

pltu.set_title("Utilit\\'e")

pltce.plot(ps,errorce[:,2],label="Information concentr\\'ee")
pltce.axis(ymax=0.18)
pltce.set_yticks([0,0.02,0.04,0.06,0.08,0.1])
pltce.tick_params('y',colors='C0')

pltce2 = pltce.twinx()
pltce2.plot(ps,boundce[:,2],'k--')
pltce2.axis(ymin=-400)
pltce2.set_yticks([0,100,200,300,400])

pltce.set_xlabel('$p$')
pltce2.set_title("\\'Equivalent certain")

plt.savefig('fig/pconst_so_auc.pdf',bbox_inches='tight',pad_inches=0)


In [307]:
plt.figure(figsize=(10,20))
fig,ax = plt.subplots(2,sharex=True,figsize=(6,8))
pltu,pltce = ax
pltu.plot(ps,error[:,1],label="Information dispers\\'ee")
pltu.plot(ps[0],error[0,1],'k--',label='Borne')
yo = pltu.get_yticks()
pltu.set_yticks([el for el in yo if el <= error[:,1].max()])
pltu.tick_params('y',colors='C0')
pltu.axis(ymax=0.7,ymin=0)


pltu2 = pltu.twinx()
pltu2.plot(ps,bound[:,1],'k--')
pltu2.axis(ymin=-300)
yo = pltu2.get_yticks()
pltu2.set_yticks([el for el in yo if el >= bound[:,1].min()]+[0])
pltu.legend(loc='upper left')

pltu.set_title("Utilit\\'e")

pltce.plot(ps,errorce[:,1],label="Information concentr\\'ee")
pltce.axis(ymax=0.85)
pltce.set_yticks([0,0.1,0.2,0.3,0.4])
pltce.tick_params('y',colors='C0')

pltce2 = pltce.twinx()
pltce2.plot(ps,boundce[:,1],'k--')
pltce2.axis(ymin=-500)
pltce2.set_yticks([0,100,200,300,400,500])

pltce.set_xlabel('$p$')
pltce2.set_title("\\'Equivalent certain")

# plt.savefig('fig/pconst_so_disp.pdf',bbox_inches='tight',pad_inches=0)


In [0]:
plt.figure(figsize=(10,20))
fig,ax = plt.subplots(2,sharex=True,figsize=(6,8))
pltu,pltce = ax
pltu.plot(ps,error[:,0],label="Information concentr\\'ee")
pltu.plot(ps[0],error[0,0],'k--',label='Borne')
pltu.axis(ymax=0.7)
yo = pltu.get_yticks()
pltu.set_yticks([el for el in yo if el <= error[:,0].max()])
pltu.tick_params('y',colors='C0')

pltu2 = pltu.twinx()
pltu2.plot(ps,bound[:,0],'k--')
pltu2.axis(ymin=-50)
yo = pltu2.get_yticks()
pltu2.set_yticks([el for el in yo if el >= bound[:,0].min()]+[300])
pltu.legend(loc='upper left')

pltu.set_title("Utilit\\'e")

pltce.plot(ps,errorce[:,0],label="Information concentr\\'ee")
pltce.axis(ymin=20,ymax=50)
pltce.set_yticks([20,22,24])
pltce.tick_params('y',colors='C0')

pltce2 = pltce.twinx()
pltce2.plot(ps,boundce[:,0],'k--')
pltce2.set_yscale('log')

pltce.set_xlabel('$p$')
pltce2.set_title("\\'Equivalent certain")

# plt.savefig('fig/pconst_so_conc.pdf',bbox_inches='tight',pad_inches=0)


In [275]:
boundce[:,0]

array([  1.42863021e+13,   4.28725867e+12,   4.37883883e+12,
         4.47117316e+12,   5.63517963e+11,   5.74802283e+11,
         5.86002231e+11,   3.25585441e+12,   2.14521734e+12,
         6.19214397e+11,   4.09401105e+12,   6.41148828e+11,
         5.04998377e+12,   2.87753133e+12,   7.82151354e+11,
         2.83983624e+12,   8.07288445e+11,   3.05758881e+12,
         1.33697660e+13,   7.27743863e+11,   3.19252031e+12,
         4.06188121e+12,   1.41745262e+13,   2.71837182e+12,
         1.33496147e+13,   6.04732566e+12,   1.11683609e+13,
         8.13207114e+11,   8.23810149e+11,   4.59428066e+12,
         8.44985407e+11,   7.61131076e+12,   3.05564623e+12,
         2.76757538e+12,   8.87311447e+11,   8.27663028e+12,
         1.97435157e+12,   3.90660194e+12,   3.94843040e+12,
         9.40019669e+11,   3.53900351e+12,   1.07919056e+13,
         1.66729829e+13,   4.15495258e+12,   1.38441841e+13,
         4.23385615e+12,   1.01339591e+12,   3.81248270e+12,
         3.64608179e+12,

In [267]:
plt.plot(boundce)

## Utilité et utilité inverse

In [0]:
us = [rnu(),leu(1),leu(0.5),leu(0.25)]
rs = np.linspace(-0.5,1,50)

fig,ax = plt.subplots(1,figsize=(6,6))
ax.set_aspect('equal')
ax.plot(rs,rnu()(rs),'C0',label='$\mu = \infty$')
ax.plot(rs,leu(1)(rs),'C0-.',label='$\mu = 1$')
ax.plot(rs,leu(0.5)(rs),'C0--',label='$\mu=1/2$')
ax.plot(rs,leu(0.05)(rs),'C0:',label='$\mu=5/100$')
ax.legend()
ax.spines['left'].set_position('zero')
ax.spines['left'].set_alpha(0.5)
ax.spines['right'].set_color('none')
ax.spines['bottom'].set_position('zero')
ax.spines['bottom'].set_alpha(0.5)
ax.spines['top'].set_color('none')

ax.annotate('$r$', xy=(0.9,-0.1))
ax.annotate('$u$', xy=(0.05,0.9))
plt.savefig('fig/leus.pdf',bbox_inches='tight',pad_inches=0)


In [0]:
u = leu(1)
rs = np.linspace(-0.1,1.6,100)
fig,ax = plt.subplots(1)
ax.plot(rs[np.where(rs<0.75)],u.inverse(rs[np.where(rs<0.75)]),'w',linewidth=8,zorder=2)
ax.plot(rs,u.inverse(rs),'k--',label='$LEU_1^{-1}$',zorder=2)
ax.plot(rs[np.where(rs>0.75)],u(rs[np.where(rs>0.75)]),'w',linewidth=8,zorder=2)
ax.plot(rs,u(rs),'C0',label='$LEU_1$',zorder=5)
ax.axvline(1,color='k',linestyle=':',alpha=0.5,zorder=1)
ax.axhline(1,color='C0',linestyle=':',alpha=0.5,zorder=1)

ax.legend()
ax.axis(ymax=4)


ax.spines['left'].set_position('zero')
ax.spines['left'].set_alpha(0.5)
ax.spines['right'].set_color('none')
ax.spines['bottom'].set_position('zero')
ax.spines['bottom'].set_alpha(0.5)
ax.spines['top'].set_color('none')

plt.savefig('fig/leu_inv.pdf',bbox_inches='tight',pad_inches=0)


/Users/TBM/Recherche/Memoire/cd/model/utility.py:55: RuntimeWarning: invalid value encountered in log
  return np.where(r<=0,r,-b*np.log(1-r/b))


## Erreur de généralisation (n=10)

In [8]:
u = leu(1)
bar_p = 50
ps = np.arange(1,p_bar+1)
R = RademacherDistribution()
Xs = [RademacherDistribution() for _ in range(bar_p)]

In [10]:
corvecs = np.zeros((3,bar_p))
corvecs[0] = 0.95/np.sqrt(bar_p) * np.ones(bar_p)
corvecs[1,0] = 0.95

In [49]:
corvecs[1]

array([ 0.95,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ])

In [12]:
Ms = [synth.GaussianMarket(Xs,R,corr_vector=corvec) for corvec in corvecs]
t_trues = [M.sample_t(5_000) for M in Ms]

In [200]:
m = 150
n = 10
tss = [create_sample(t_true,n=n,m=m) for t_true in t_trues]

In [201]:
k = 3
error = np.zeros((bar_p,k))
bound = np.zeros((bar_p,k))
errorce = np.zeros((bar_p,k))
boundce = np.zeros((bar_p,k))
compin = np.zeros((bar_p,k))
compout = np.zeros((bar_p,k))
for i,p in enumerate(ps):
    if i%5 == 0: print(i)
    for j,(ts,t_true) in enumerate(zip(tss,t_trues)):
        t = ts[:,:,:p]
        qs = solve(t,u,lamb=1)
        inerror = in_error(qs,u,t,lamb=1)
        outerror = out_error(qs,u,t_true,lamb=1)
        error[i,j] = np.percentile(inerror - outerror,95)
        errorce[i,j] = np.percentile(u.inverse(inerror) - u.inverse(outerror),95)
        bound[i,j] = bound_genu(n,sqrt(p),lamb=1)
        boundce[i,j] = bound[i,j]*u.subinverse(errorce[i,j])
        idx = np.argmin(np.abs(error[i,j] - (inerror - outerror)))
        compin[i,j] = inerror[idx]
        compout[i,j] = outerror[idx]

45


40


35


30


25


20


15


10


5


0


In [202]:
fig,axarr = plt.subplots(2,figsize=(6,8))
fig.subplots_adjust(hspace=.3)
pltu,pltce = axarr

pltu.set_title('a) Utilité')
pltu.plot(ps,bound,'k--')
pltu.axis(ymin=-80)
pltu.yaxis.set_label_position('left')
pltu.set_yticks(np.arange(0,90,10))
pltu.set_xlabel('$p$')
pltu.set_ylabel('Erreur de généralisation (util)')

pltu = pltu.twinx()
pltu.plot(ps[0],error[0,0],'k--',label='Borne théorique $(\delta=5\%)$')
pltu.plot(ps,error[:,2],'C0-o',markevery=4,ms=4,label='Aucune information')
pltu.plot(ps,error[:,0],'C0',markevery=4,ms=4,label='Information dispersée')
pltu.plot(ps,error[:,1],'C0-s',markevery=4,ms=4,label='Information concentrée')
pltu.legend(loc='upper left')
pltu.axis(ymin=0,ymax=1.4)
pltu.set_xticks([1]+list(range(5,55,5)));
pltu.set_yticks(np.arange(0,0.91,0.1))
pltu.tick_params('y',colors='C0')

pltce.set_title('a) Équivalent certain')
pltce.plot(ps,boundce[:,2],'k--o',markevery=4,ms=4,label='Aucune information')
pltce.plot(ps,boundce[:,0],'k--',label='Information dispersée')
pltce.plot(ps,boundce[:,1],'k--s',markevery=4,ms=4,label='Information concentrée')
pltce.axis(ymin=-400)
pltce.yaxis.set_label_position('left')
pltce.set_yticks(np.arange(0,500,100))
pltce.set_xlabel('$p$')
pltce.set_ylabel('Erreur de généralisation (Équivalent certain)')

pltce = pltce.twinx()
# pltce.plot(ps[0],errorce[0,0],'k--',label='Borne théorique $(\delta=5\%)$')
pltce.plot(ps[0],errorce[0,2],'k-o',markevery=4,ms=4,label='Borne théorique $(\delta = 5\%)$')
pltce.plot(ps[0],errorce[0,0],'k',markevery=4,ms=4,label='Borne théorique $(\delta = 5\%)$')
pltce.plot(ps[0],errorce[0,1],'k-s',markevery=4,ms=4,label='Borne théorique $(\delta = 5\%)$')
pltce.plot(ps,errorce[:,2],'C0-o',markevery=4,ms=4,label='Aucune information')
pltce.plot(ps,errorce[:,0],'C0',markevery=4,ms=4,label='Information dispersée')
pltce.plot(ps,errorce[:,1],'C0-s',markevery=4,ms=4,label='Information concentrée')

pltce.legend(loc='upper left',ncol=2)
pltce.axis(ymin=0,ymax=3)
pltce.set_yticks(np.arange(0,1.51,0.25))
pltce.set_xticks([1]+list(range(5,55,5)));
pltce.tick_params('y',colors='C0')

# plt.savefig('fig/pconst_infogen2.pdf',bbox_inches='tight',pad_inches=0)


In [240]:
plt.plot(ps,error)

In [220]:
def order(x,a,k,b):
    return a*x**k + b

def order12(x,a,b):
    return a*x**(1/2) + b

def order112(x,a0,a1,b):
    return a0*x + a1*x**(1/2) + b

In [249]:
c = 1
popt,_ = curve_fit(order112,ps[-25:],error[-25:,c])
plt.plot(ps,error[:,c])
plt.plot(ps,order112(ps,*popt))



## Polyfit

In [251]:
def order(x,a,k,b):
    return a*x**k + b

def order12(x,a,b):
    return a*x**(1/2) + b

def order112(x,a0,a1,b):
    return a0*x + a1*x**(1/2) + b

In [274]:
titles = ['Information dispersée','Information concentrée','Aucune information']
fig,ax = plt.subplots(3,1,figsize=(6,9))
fig.subplots_adjust(hspace=.35)
for c,cp in enumerate(ax):
    cp.plot(ps,error[:,c])
    popt,_ = curve_fit(order12,ps[-25:],error[-25:,c])
    cp.plot(ps,order12(ps,*popt),'k:',alpha=0.5,label='Hypothèse $O(p^{1/2})$')
    popt,_ = curve_fit(order112,ps[-25:],error[-25:,c])
    cp.plot(ps,order112(ps,*popt),'k--',alpha=0.5,label='Hypothèse $O(p+p^{1/2})$')
    cp.axis(ymin=0,ymax=1)
    cp.legend(loc='lower right')
    cp.set_xticks([1]+list(range(5,55,5)));
    cp.set_xlabel('$p$')
    cp.set_title(titles[c])

plt.savefig('fig/pconst_infogen_cf.pdf',bbox_inches='tight',pad_inches=0)

## Progression de EU*

In [317]:
Ms = [synth.GaussianMarket(Xs,R,corr_vector=corvec) for corvec in corvecs]
t_trues = [M.sample_t(5_000) for M in Ms]

In [318]:
q_trues = [[solve(t_true[:,:p],u=u,lamb=0)[0] for p in ps] for t_true in t_trues]
q_trues = np.array([np.array(list(zip_longest(*q_true,fillvalue=0))).T for q_true in q_trues])

In [319]:
eu_trues = np.array([[u(t_true@q_true).mean() for q_true in q_true0] for q_true0,t_true in zip(q_trues,t_trues)])

In [330]:
fig,ax = plt.subplots(1)
ax.plot(ps,eu_trues[1,:],'C0',label="Information concentrée")
ax.plot(ps,eu_trues[0,:],'C0-s',ms=4,markevery=4,label="Information dispersée")
ax.plot(ps,eu_trues[2,:],'C0-o',ms=4,markevery=4,label='Aucune information')
ax.axis(ymax=0.9)
ax.set_xticks([1]+list(range(5,55,5)));
ax.legend()
ax.set_xlabel('$p$')
ax.set_ylabel('Utilité')
# plt.savefig('fig/pconst_euinforelative.pdf',bbox_inches='tight',pad_inches=0)


In [347]:
m = 150
n = 50
tss = [create_sample(t_true,n=n,m=m) for t_true in t_trues]

In [348]:
k = 3
error = np.zeros((len(ps),k))
errorce = np.zeros((len(ps),k))
compout = np.zeros((len(ps),k))
for i,p in enumerate(ps):
    if i%5 == 0: print(i)
    for j,(ts,t_true,eu_true) in enumerate(zip(tss,t_trues,eu_trues)):
        t = ts[:,:,:p]
        qs = solve(t,u,lamb=1)
        outerror = out_error(qs,u,t_true,lamb=1)
        errorce[i,j] = np.percentile(u.inverse(eu_true[j]) - u.inverse(outerror),95)
        error[i,j] = np.percentile(eu_true[j] - outerror,95)
        idx = np.argmin(np.abs((eu_true[j] - outerror) - error[i,j]))
        compout[i,j] = outerror[idx]

45


40


35


30


25


20


15


10


5


0


In [349]:
plt.plot(ps,compout)

In [350]:
fig,axarr = plt.subplots(2,sharex=True,figsize=(6,8))
pltu,pltce = axarr
pltu.plot(ps,error[:,1],'C0',label="Information concentrée")
pltu.plot(ps,error[:,0],'C0-s',ms=4,markevery=4,label="Information dispersée")
pltu.plot(ps,error[:,2],'C0-o',ms=4,markevery=4,label='Aucune information')
pltu.axis(ymax=0.7)
pltu.set_title("a) Utilité")
pltu.legend(loc='upper left')

# 
pltce.plot(ps[0],errorce[0,2],'C0',ms=4,markevery=4,label="Info. concentr\\'ee")
pltce.plot(ps,errorce[:,1],'C0-s',ms=4,markevery=4,label="Info. dispers\\'ee")
pltce.plot(ps,errorce[:,2],'C0-o',ms=4,markevery=4,label='Aucune information')
pltce.set_title("b) Équivalent certain")
pltce.set_xlabel('$p$')
pltce.axis(ymax=1.2)
# pltce.tick_params('y',colors='C0')

yo = pltce.get_yticks()
pltce.set_yticks([el for el in yo if el <= errorce[:,1].max()])
pltce.axis(ymin=-0.1)
# pltce2 = pltce.twinx()
# pltce2.plot(ps,errorce[:,0],'C0')
# pltce2.axis(ymax=21.96,ymin=21.5)
# yo = pltce2.get_yticks()
# pltce2.set_yticks([el for el in yo if el >= errorce[:,0].min()]+[21.95,21.85,21.8]);

pltce.legend(loc='upper left')

# plt.savefig('fig/pconst_infosototale.pdf',bbox_inches='tight',pad_inches=0)

